In [1]:
!wget -O data.zip "https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip"

--2022-01-09 10:28:18--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.44.26
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.44.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95592747 (91M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  91.16M  25.6MB/s    in 3.6s    

2022-01-09 10:28:22 (25.6 MB/s) - ‘data.zip’ saved [95592747/95592747]



In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: dataset2.zip            


In [ ]:
!unzip dataset2.zip

In [6]:
import os, shutil, glob

In [9]:
types = [ ("Cloudy","cloud*.*"), ("Rainy","rain*.*"), ("Shiny","shin*.*"), ("Sunrise","sunrise*.*")  ]

shutil.rmtree("dataset")
for folder_name, file_mask in types:
  files = sorted(glob.glob("dataset2/"+file_mask))
  os.makedirs(f"dataset/Train/{folder_name}")
  os.makedirs(f"dataset/Test/{folder_name}")

  train_qnty = int(0.8*len(files))

  [shutil.copy(item, f"dataset/Train/{folder_name}") for item in files[:train_qnty] ]
  [shutil.copy(item, f"dataset/Test/{folder_name}") for item in files[train_qnty:] ]
  print(folder_name, file_mask, len(files))

Cloudy cloud*.* 300
Rainy rain*.* 215
Shiny shin*.* 253
Sunrise sunrise*.* 357


In [10]:
!wget -O cup.jpeg "https://lh5.googleusercontent.com/sTDIn5BK6czLJsB9u-m-VcLma4a6IbJ6rKL0NC3Vp8XESJuAoc5detOT9wtNobOX2CvCX4Qtel21vr7R3Jnv13MrbNIy-yhb-4z6H08wIthMXhce8mjglvufpLOzgx0ftFbvGphn"

--2022-01-09 10:46:41--  https://lh5.googleusercontent.com/sTDIn5BK6czLJsB9u-m-VcLma4a6IbJ6rKL0NC3Vp8XESJuAoc5detOT9wtNobOX2CvCX4Qtel21vr7R3Jnv13MrbNIy-yhb-4z6H08wIthMXhce8mjglvufpLOzgx0ftFbvGphn
Resolving lh5.googleusercontent.com (lh5.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to lh5.googleusercontent.com (lh5.googleusercontent.com)|173.194.210.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38944 (38K) [image/jpeg]
Saving to: ‘cup.jpeg’

cup.jpeg            100%[===================>]  38.03K  --.-KB/s    in 0s      

2022-01-09 10:46:41 (95.4 MB/s) - ‘cup.jpeg’ saved [38944/38944]



In [29]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.layers import Flatten, Dense
from keras.models import Model

In [15]:
# łądowanie grafiki
img = load_img("cup.jpeg", target_size=(224,224) )

# konwersja na tablice NumPy
img_arr = img_to_array(img)

arr = np.expand_dims(img_arr, axis=0)
# konwersja na potrzeby sieci
arr = preprocess_input(arr)

In [17]:
model = VGG16()
prediction = model.predict(arr)

In [20]:
decode_predictions(prediction)[0]

[('n03063599', 'coffee_mug', 0.25186822),
 ('n07930864', 'cup', 0.17223488),
 ('n04263257', 'soup_bowl', 0.16073191),
 ('n07920052', 'espresso', 0.14333373),
 ('n03733805', 'measuring_cup', 0.08206768)]

In [ ]:
model.summary()

In [30]:
# Learning Transfer z sieci VGG16
model = VGG16(include_top=False, input_shape=(224,224,3))
for layer in model.layers:
  layer.trainable = False

hidden = Flatten()(model.output)
outputs = Dense(4, activation="softmax")(hidden)

# tworzenie nowej sieci na podstawie instniejącej
model = Model(inputs=model.input, outputs=outputs)

# zobacz strukture
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [31]:
# kompilacja
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
# Augemntacja danych dla sieci

train_datagen = ImageDataGenerator(    
    rotation_range = 40, zoom_range = 0.2,
    shear_range = 0.2, horizontal_flip = True, vertical_flip = True,
    width_shift_range = 0.2,
    rescale = 1/255
)
train_generator = train_datagen.flow_from_directory(directory="dataset/Train", target_size=(224,224), class_mode="categorical")

test_datagen = ImageDataGenerator(
    rescale = 1/255
)
test_generator = test_datagen.flow_from_directory(directory="dataset/Test", target_size=(224,224), class_mode="categorical")

Found 899 images belonging to 4 classes.
Found 226 images belonging to 4 classes.


In [ ]:
model.fit_generator(train_generator, validation_data=test_generator, epochs=5)